In [5]:
import openai, json
from functools import reduce  # forward compatibility for Python 3
import operator
openai_key = open('../open_ai.key','r').read()
openai.api_key = openai_key

In [6]:
def get_from_dict(dataDict, mapList):
    return reduce(operator.getitem, mapList, dataDict)

def set_in_dict(dataDict, mapList, message):
    get_from_dict(dataDict, mapList[:-1])[mapList[-1]] = {'text': message}
    return dataDict

def shunted_chat(chat: dict, messages: dict) -> dict:
    """
    対話APIを使って、対話が分流するようなPythonプログラム
    """
    current = chat
    past_messages = []
    for key in messages['keys']:
        current = current[key]
        past_messages.append(current['text'])
    # print('current:',current)
    past_messages.append({'role':'user','content':messages['text']})
    q = 'Q' + str(len(current.keys()))
    messages['keys'].append(q)
    chat = set_in_dict(chat, messages['keys'], {'role':'user','content':messages['text']})
    current = current[q]
    
    # for m in past_messages:
    #     print(m)
    result = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=past_messages,
        max_tokens=100,
    )
    # print('res:',json.dumps(result,indent=2))
    messages['keys'].append('A')
    chat = set_in_dict(chat, messages['keys'], result.choices[0]['message'])
    # print(messages['keys'])
    # print(json.dumps(chat,indent=2))

    # messages = [chat[k]['text']['content'] for k in messages['keys'] if ]
    chat_history = []
    current = chat
    for key in messages['keys']:
        chat_history.append(current[key]['text']['role']+': '+current[key]['text']['content'].replace('\n\n\n','\n').replace('\n\n','\n'))
        current = current[key]
    return chat,chat_history

In [7]:
# chat = {
#     'setting': {'text': {'role': 'assistant', 'content': 'Hello.'}},
#     'Q1': {
#         'text': {'role': 'user', 'content': 'Hi, how are you?'},
#         'A': {
#             'text': {'role': 'assistant', 'content': 'I am fine, thank you.'},
#         }
#     }
# }

# chat,history = shunted_chat(chat, {'keys': ['Q1', 'A'], 'text': 'What is your name?'})

# for m in history:
#     print(m)

In [8]:
chat = {'setting': {'text': {'role': 'system', 'content': 'あなたは料理研究家です。'}}}
chat,history = shunted_chat(chat, {'keys': [], 'text': 'さつまいもの調理方法を教えてください。'})
print('\n'.join(history))
# print(json.dumps(chat,indent=2,ensure_ascii=False))

user: さつまいもの調理方法を教えてください。
assistant: 
さつまいもを調理する方法は、以下の通りです。
1. さつまいもを洗う：さつまいもを水でよく洗い、汚れや泥を落とします。
2. 蒸す：さつまいもを蒸し器に入れ、15～20分程度蒸します。蒸し器がない


In [9]:
chat,history = shunted_chat(chat, {'keys': [], 'text': 'じゃがいもの調理方法を教えてください。'})
print('\n'.join(history))
# print(json.dumps(chat,indent=2,ensure_ascii=False))

user: じゃがいもの調理方法を教えてください。
assistant: 
1. 煮る方法：
- じゃがいもを洗って皮をむく。
- じゃがいもを適当な大きさに切る。
- 鍋に水を入れて沸騰させ、そこにじゃがいもを入れる。
- じゃがいもが柔らかくなるまで10


In [10]:
chat,history = shunted_chat(chat, {'keys': ['Q1','A'], 'text': 'レシピを一つを教えてください。'})
print('\n'.join(history))

user: さつまいもの調理方法を教えてください。
assistant: 
さつまいもを調理する方法は、以下の通りです。
1. さつまいもを洗う：さつまいもを水でよく洗い、汚れや泥を落とします。
2. 蒸す：さつまいもを蒸し器に入れ、15～20分程度蒸します。蒸し器がない
user: レシピを一つを教えてください。
assistant: 「さつまいもの甘煮」
【材料】
・さつまいも 2本
・砂糖 100g
・水 200ml
・しょうが（1cm程度） 1枚
・薄口しょうゆ 大さじ1
・黒ゴマ 適量
【作り方】
1.


In [11]:
chat,history = shunted_chat(chat, {'keys': ['Q2','A'], 'text': 'レシピを一つを教えてください。'})
print('\n'.join(history))

user: じゃがいもの調理方法を教えてください。
assistant: 
1. 煮る方法：
- じゃがいもを洗って皮をむく。
- じゃがいもを適当な大きさに切る。
- 鍋に水を入れて沸騰させ、そこにじゃがいもを入れる。
- じゃがいもが柔らかくなるまで10
user: レシピを一つを教えてください。
assistant: ポテトサラダのレシピをお教えします。
【材料】
・じゃがいも　４個
・にんじん　１本
・きゅうり　１/２本
・玉ねぎ　１/４個
・ツナ缶　１缶
・マヨネーズ　大さじ３
・塩　


In [12]:
def print_chat(d, indent=0):
    for key, value in d.items():
        # print(key,value)
        if isinstance(value, dict):
            if 'text' in value.keys():
                role = value['text']['role']
                content = value['text']['content']
                print(' ' * indent + key + ' ' + role + ': ' + content.replace('\n\n','\n').replace('\n','\n'+' ' * (indent+1)))
                value.pop('text')
                print_chat(value, indent+4)
            else:
                print('?',value)
        else:
            print('?',value)
            role = value['role']
            content = value['content']
            print(' ' * indent + key + ' ' + role + ': ' + content)

print_chat(chat)


setting system: あなたは料理研究家です。
Q1 user: さつまいもの調理方法を教えてください。
    A assistant: 
     さつまいもを調理する方法は、以下の通りです。
     1. さつまいもを洗う：さつまいもを水でよく洗い、汚れや泥を落とします。
     2. 蒸す：さつまいもを蒸し器に入れ、15～20分程度蒸します。蒸し器がない
        Q1 user: レシピを一つを教えてください。
            A assistant: 「さつまいもの甘煮」
             【材料】
             ・さつまいも 2本
             ・砂糖 100g
             ・水 200ml
             ・しょうが（1cm程度） 1枚
             ・薄口しょうゆ 大さじ1
             ・黒ゴマ 適量
             【作り方】
             1.
Q2 user: じゃがいもの調理方法を教えてください。
    A assistant: 
     1. 煮る方法：
     - じゃがいもを洗って皮をむく。
     - じゃがいもを適当な大きさに切る。
     - 鍋に水を入れて沸騰させ、そこにじゃがいもを入れる。
     - じゃがいもが柔らかくなるまで10
        Q1 user: レシピを一つを教えてください。
            A assistant: ポテトサラダのレシピをお教えします。
             【材料】
             ・じゃがいも　４個
             ・にんじん　１本
             ・きゅうり　１/２本
             ・玉ねぎ　１/４個
             ・ツナ缶　１缶
             ・マヨネーズ　大さじ３
             ・塩　
